# Formula

From known system of linear equation, it will try to convert into augmented matrix form: 
$$[A|b]$$

Then, the goal is to make the matrix AA into an upper triangular form:

$$
\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n} & b_1 \\
0 & a_{22}' & \cdots & a_{2n}' & b_2' \\
0 & 0 & \cdots & a_{3n}' & b_3' \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & \cdots & a_{nn}' & b_n'
\end{bmatrix}
$$

In [47]:
# Install necessary library
import numpy as np
from ipywidgets import VBox, HBox, IntText, Button, Output, RadioButtons
import matplotlib.pyplot as plt

---
# Manual

In [49]:
def solve_manually(coefficients, constants):
    A = np.array([[coeff.value for coeff in row] for row in coefficients], dtype=float)
    B = np.array([const.value for const in constants], dtype=float)
    
    n = len(B)
    
    for i in range(n):
        if A[i, i] == 0:
            raise ValueError(f"Zero pivot encountered at row {i}")
        
        for j in range(i + 1, n):
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            B[j] -= factor * B[i]
    
    solution = np.zeros(n)
    for i in range(n - 1, -1, -1):
        solution[i] = (B[i] - np.dot(A[i, i + 1:], solution[i + 1:])) / A[i, i]
        
    return solution

---
# Easy way
Use `numpy.linalg.solve()` function to easily find the solution

In [38]:
def solve_with_np_linalg_solve(coefficients, constants):
    A = np.array([[coeff.value for coeff in row] for row in coefficients])
    B = np.array([const.value for const in constants])
            
    return np.linalg.solve(A, B)

---
# Testing

In [48]:
output = Output()

size_input = IntText(value=0, description="Matrix size:")
generate_button = Button(description="Generate Form", button_style='info')
solve_button = Button(description="Solve", button_style='success', disabled=True)
method_selection = RadioButtons(options=['Manual', 'numpy.linalg.solve()'], description='Select method')
matrix_area = VBox()

def generate_form(_):
    n = size_input.value
    
    coefficients = [[IntText(value=0, description=f"a{i+1}{j+1}") for j in range(n)] for i in range(n)]
    constants = [IntText(value=0, description=f"b{i+1}") for i in range(n)]
    
    rows = [HBox(row + [constants[i]]) for i, row in enumerate(coefficients)]
    form = VBox(rows)
    
    solve_button.coefficients = coefficients
    solve_button.constants = constants
    solve_button.disabled = False
    
    matrix_area.children = [form]
    with output:
        output.clear_output()

def solve_linear_system(_):
    coefficients = solve_button.coefficients
    constants = solve_button.constants
    
    solution = None
    
    with output:
        output.clear_output()
        try:
            if method_selection.value == 'Manual':
                solution = solve_manually(coefficients, constants)
            elif method_selection.value == 'numpy.linalg.solve()':
                solution = solve_with_np_linalg_solve(coefficients, constants)
                
            print("Solution:")
            for i, x in enumerate(solution, start=1):
                print(f"x{i} = {x}")
        except Exception as e:
            print(f"Error: {e}")
    
            

generate_button.on_click(generate_form)
solve_button.on_click(solve_linear_system)

layout = VBox([
    size_input,
    generate_button,
    matrix_area,
    method_selection,
    solve_button,
    output
])
layout